In [2]:
# Installing necessary packages
!pip install earthengine-api
!pip install rasterio
!pip install google-colab
!pip install pyarrow
!pip install folium
!pip install pyproj

In [117]:
import ee

ee.Authenticate()
ee.Initialize(project='mse-uhi')

Extracting Image Data (Singapore)

In [42]:
import ee
ee.Initialize()

# Define the region of interest
singapore = ee.Geometry.Rectangle([103.59, 1.13, 104.07, 1.47])

# Load the Landsat 8 Collection 2 Tier 2 Image Collection
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

year = "2023"

# Filter by date and location for a specific year
filtered_collection = Landsat8_collection.filterDate(f'{year}-01-01', f'{year}-12-31').filterBounds(singapore)

image_list = filtered_collection.toList(filtered_collection.size())
print(image_list.size().getInfo())

def export_each(image):
    # Format date for file naming
    date_time = image.date().format('YYYYMMdd_HHmmss').getInfo()
    
    # Select bands, assuming we need all available bands
    image_selected = image.toFloat()  # Convert to float if needed for processing

    # Export each image to Drive
    task = ee.batch.Export.image.toDrive(
        image=image_selected,
        description=f'L8_UTC_{date_time}',
        folder=f'L8_Data/{year}',
        scale=30,
        region=singapore.getInfo()['coordinates'],
        fileFormat='GeoTIFF'
    )
    task.start()
    print(f"Task {task.id} started for {date_time}. Checking status...")

    # Check and print the status of the task
    while task.active():
        task_status = task.status()
        print(f"Task {task.id} for {date_time}: {task_status['state']}")
        if task_status['state'] == 'COMPLETED':
            print(f"Task {task.id} for {date_time} completed successfully.")
            break
        elif task_status['state'] == 'FAILED':
            print(f"Task {task.id} for {date_time} failed with error: {task_status['error_message']}")
            break

# Apply the function to each image in the collection
for i in range(image_list.size().getInfo()):
    image = ee.Image(image_list.get(i))
    export_each(image)

Task FTG4ADVU3P3KY63TKATXQPTB started for 20220111. Checking status...
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: READY
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: READY
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNING
Task FTG4ADVU3P3KY63TKATXQPTB for 20220111: RUNNI

In [13]:
import rasterio
import pandas as pd
import numpy as np
import os

year = "2022"
data_directory = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\{year}"

# Loop through each file in the directory
for filename in os.listdir(data_directory):
    prefix = f"L8_UTC_{year}"
    if filename.startswith(prefix):
        file_path = os.path.join(data_directory, filename)

        # Open the GeoTIFF file
        with rasterio.open(file_path) as src:
            array = src.read()
            nodata_values = src.nodatavals
            band_names = src.descriptions

            # Reshape the array to a 2D array where each row is a pixel
            reshaped_array = array.transpose(1, 2, 0).reshape(-1, len(band_names))
            bands_df = pd.DataFrame(reshaped_array, columns=band_names)
            bands_df['x'] = np.tile(np.arange(src.width), src.height)
            bands_df['y'] = np.repeat(np.arange(src.height), src.width)
            bands_df['ST_B10_Celsius'] = bands_df['ST_B10'] * 0.00341802 + 149 - 273.15
            
            # Scaling and offsetting specific bands
            bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
            for band in bands_to_scale:
                bands_df[f"{band}_Scaled"] = bands_df[band] * 2.75e-05 - 0.2

            additional_scales = {
                'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
                'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
                'ST_TRAD': 0.001, 'ST_URAD': 0.001
            }
            for band, scale in additional_scales.items():
                bands_df[f"{band}_Scaled"] = bands_df[band] * scale

            # Replace NoData values appropriately
            for i, nodata_value in enumerate(nodata_values):
                if nodata_value is not None:
                    bands_df.iloc[:, i].replace(nodata_value, np.nan, inplace=True)

            # Save the bands data to CSV
            bands_csv_path = os.path.join(data_directory, f'Landsat8_Bands_{filename[-19:-4]}.csv')
            bands_df.to_csv(bands_csv_path, index=False)

            print(f"Saved band data to {bands_csv_path}") 

            # Extract and save metadata to CSV
            metadata = src.meta
            metadata_df = pd.DataFrame([metadata])
            metadata_csv_path = os.path.join(data_directory, f'Landsat8_Metadata_{filename[-19:-4]}.csv')
            metadata_df.to_csv(metadata_csv_path, index=False)

            print(f"Saved metadata to {metadata_csv_path}") 

Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Bands_20220111_031652.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Metadata_20220111_031652.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Bands_20220127_031646.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Metadata_20220127_031646.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Bands_20220212_031643.csv
Saved metadata to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Metadata_20220212_031643.csv
Saved band data to C:\LocalOneDrive\Documents\Desktop\MTI\UHI-Project\MSE-ES-UHI\Data\Landsat8\2022\Landsat8_Bands_20220228_031639.csv
Saved metadata to C:\LocalOneDrive\Documents\Desk